In [ ]:
import pandas as pd
import numpy as np
import datetime
import plotly.graph_objects as go
import warnings; warnings.simplefilter('ignore')
from IPython.display import display, Markdown, Image, SVG
import re
import bmondata
from bmondata import Server
from dateutil.relativedelta import relativedelta
import scrapbook as sb

In [ ]:
# Parameters to be changed/exported using Papermill or Scrapbook
org_id = 1
server_web_address = 'https://bms.ahfc.us'

In [ ]:
# 'Glue' down variables for scrapbook to export later
sb.glue('sort_order', 6)
sb.glue('title', 'Org. Energy Comp')

server = Server(server_web_address)

# Write test to make sure there is more than one building for comparison.

def calc_monthly_elec_cost(electric_rates, monthly_kwh, fifteen_min_peak):
        if electric_rates['block1'] is None:
            monthly_usage_cost = electric_rates['customer_charge'] + electric_rates['rate1'] * monthly_kwh
            monthly_demand_charges = electric_rates['demand_charge'] * fifteen_min_peak
            total_monthly_cost = monthly_usage_cost + monthly_demand_charges
        elif isinstance(electric_rates['block1'], float):
                if monthly_kwh > electric_rates['block1']: 
                    block_1_cost = electric_rates['block1'] * electric_rates['rate1']
                    remaining_usage = monthly_kwh - electric_rates['block1']
                    remainder_cost = remaining_usage * electric_rates['rate2']
                    monthly_usage_cost = electric_rates['customer_charge'] + block_1_cost + remainder_cost
                    monthly_demand_charges = electric_rates['demand_charge'] * fifteen_min_peak
                    total_monthly_cost = monthly_usge_cost + monthly_demand_charges
                elif monthly_kwh <= electric_rates['block1']:
                    monthly_usage_cost = electric_rates['customer_charge'] + electric_rates['rate1'] * monthly_kwh
                    monthly_demand_charges = electric_rates['demand_charge'] * fifteen_min_peak
                    total_monthly_cost = monthly_usage_cost + monthly_demand_charges
        else: 
            print ('Cost calculation error')

        return (total_monthly_cost, monthly_usage_cost, monthly_demand_charges)

def calculate_eui(server, building_id, start_date, end_date):
    
    building_name = server.buildings(building_id)[0]['title']
    building_df = server.buildings(building_id)
    electric_id = building_df[0]['electric_ids']
    
    # Get hourly average kW
    hourly_usage_building = server.sensor_readings((electric_id, 'monthly_electric_usage_kwh'),
                           start_ts =  start_date,
                           end_ts = end_date,
                           averaging='1H')
    
    # Calculate monthly total kWh usage
    monthly_usage_building = hourly_usage_building.groupby([lambda x: x.year, lambda x: x.month]).sum()
    monthly_usage_building = monthly_usage_building.reset_index()
    monthly_usage_building = monthly_usage_building.rename(columns={'level_0': 'year','level_1':'month'})
    
    ### Calculate the hourly fuel usage rate
    fuel_ids = building_df[0]['fuel_ids']
    hourly_fuel_usage = server.sensor_readings((fuel_ids, 'fuel_usage'),
                               start_ts = start_date, end_ts = end_date,
                               averaging='1H')
    
    # Calculate the monthly total fuel usage and cost
    monthly_fuel = hourly_fuel_usage.groupby([lambda x: x.year, lambda x: x.month]).sum()
    monthly_fuel['monthly_fuel_ccf'] = monthly_fuel.fuel_usage / 100000.0
    monthly_fuel = monthly_fuel.reset_index()
    monthly_fuel = monthly_fuel.rename(columns={'level_0': 'year','level_1':'month'})
    
    # Combine the fuel and electricity usage data
    monthly_total_energy = pd.merge(monthly_fuel, monthly_usage_building, how='outer',
                                      left_on=['year', 'month'], right_on=['year', 'month'])
    
    # Calculate the annual ECI, Electric EUI, and Fuel EUI
    intensity_numbers = monthly_total_energy.sum()
    intensity_numbers['building_sqft'] = server.buildings(building_id)[0]['floor_area']
    
    intensity_numbers['electric_eui'] = intensity_numbers.monthly_electric_usage_kwh / intensity_numbers.building_sqft
    intensity_numbers['fuel_eui'] = intensity_numbers.fuel_usage / 1000.0 / intensity_numbers.building_sqft
    intensity_numbers['building_name'] = building_name
    
    return intensity_numbers

def calculate_eui_eci(server, building_id, start_date, end_date):
    
    building_name = server.buildings(building_id)[0]['title']
    electric_rates = server.buildings(building_id)[0]['electric_rate']
    building_df = server.buildings(building_id)
    electric_id = building_df[0]['electric_ids']
    
    # Get hourly average kW
    hourly_usage_building = server.sensor_readings((electric_id, 'monthly_electric_usage_kwh'),
                           start_ts =  start_date,
                           end_ts = end_date,
                           averaging='1H')
    
    # Calculate monthly total kWh usage
    monthly_usage_building = hourly_usage_building.groupby([lambda x: x.year, lambda x: x.month]).sum()
    monthly_usage_building = monthly_usage_building.reset_index()
    monthly_usage_building = monthly_usage_building.rename(columns={'level_0': 'year','level_1':'month'})
    
    # Get the fifteen minute average to calculate demand chrages
    fifteen_minute_usage_building = server.sensor_readings((electric_id, 'fifteen_minute_usage_kw'),
                                                      start_ts = start_date, end_ts = end_date,
                                                      averaging = '15min')

    # Get the highest fifteen minute average kW used per month to calculate the demand charges
    grouped_fifteen_minute_demand = fifteen_minute_usage_building.groupby([lambda x: x.year, lambda x: x.month]).max()
    grouped_fifteen_minute_demand = grouped_fifteen_minute_demand.reset_index()
    grouped_fifteen_minute_demand = grouped_fifteen_minute_demand.rename(columns={'level_0': 'year',
                                                                                 'level_1':'month'})
    
    # Combine the electric demand data with the monthly electric usage data
    monthly_electric_data = pd.merge(grouped_fifteen_minute_demand, monthly_usage_building, how='outer',
                                 left_on=['year', 'month'], right_on=['year', 'month'])
    
    # Create blank columns to fill in the for loop below
    monthly_electric_data['total_electric_cost'] = np.nan
    monthly_electric_data['monthly_kwh_cost'] = np.nan
    monthly_electric_data['monthly_demand_charges'] = np.nan
    
    # Calculate the monthly electrical cost  including customer, demand, and per kWh charges
    for i in np.arange(0, monthly_electric_data.index.shape[0]):
        (monthly_electric_data.at[i,'total_electric_cost'], 
        monthly_electric_data.at[i,'monthly_kwh_cost'],
        monthly_electric_data.at[i,'monthly_demand_charges']) = calc_monthly_elec_cost(electric_rates, 
                                                                                      monthly_electric_data.at[i, 'monthly_electric_usage_kwh'],
                                                                                      monthly_electric_data.at[i, 'fifteen_minute_usage_kw'])

    ### Calculate the hourly fuel usage rate
    fuel_rates = server.buildings(building_id)[0]['fuel_rate']
    fuel_ids = building_df[0]['fuel_ids']
    hourly_fuel_usage = server.sensor_readings((fuel_ids, 'fuel_usage'),
                               start_ts = start_date, end_ts = end_date,
                               averaging='1H')
    
    # Calculate the monthly total fuel usage and cost
    monthly_fuel = hourly_fuel_usage.groupby([lambda x: x.year, lambda x: x.month]).sum()
    monthly_fuel['monthly_fuel_ccf'] = monthly_fuel.fuel_usage / 100000.0
    monthly_fuel['monthly_fuel_cost'] = monthly_fuel.monthly_fuel_ccf * fuel_rates['rate'] + fuel_rates['customer_charge']
    monthly_fuel = monthly_fuel.reset_index()
    monthly_fuel = monthly_fuel.rename(columns={'level_0': 'year','level_1':'month'})
    
    # Combine the fuel and electricity usage and cost data
    monthly_total_energy_costs = pd.merge(monthly_fuel, monthly_electric_data, how='outer',
                                      left_on=['year', 'month'], right_on=['year', 'month'])
    monthly_total_energy_costs['total_monthly_costs'] = monthly_total_energy_costs.total_electric_cost + monthly_total_energy_costs.monthly_fuel_cost
    
    # Calculate the annual ECI, Electric EUI, and Fuel EUI
    intensity_numbers = monthly_total_energy_costs.sum()
    intensity_numbers['building_sqft'] = server.buildings(building_id)[0]['floor_area']
    
    intensity_numbers['eci'] = intensity_numbers.total_monthly_costs / intensity_numbers.building_sqft
    intensity_numbers['electric_eui'] = intensity_numbers.monthly_electric_usage_kwh / intensity_numbers.building_sqft
    intensity_numbers['fuel_eui'] = intensity_numbers.fuel_usage / 1000.0 / intensity_numbers.building_sqft
    intensity_numbers['building_name'] = building_name
    
    return intensity_numbers

def calculate_unoccupied_elec_ratio(server, building_id, start_date, end_date):
    
# Need to get the ratio of occupied to unoccupied electricity usage for each building to be able to 
# make a comparison graph
    
    building_df = server.buildings(building_id)
    electric_id = building_df[0]['electric_ids']
    
    # Get hourly average kW
    hourly_usage_building = server.sensor_readings((electric_id, 'electric_usage_kwh'),
                           start_ts =  start_date,
                           end_ts = end_date,
                           averaging='1H')
    
    # Get schedule and timezone from the API data
    building_schedule = building_df[0]['schedule']
    building_timezone = building_df[0]['timezone']

    # Create a schedule object using Ian's library
    schedule_object = bmondata.Schedule(building_schedule, building_timezone)

    if schedule_object is None:
        night_to_day_elec_use_ratio = np.nan
    elif building_schedule == '':
        night_to_day_elec_use_ratio = np.nan
    else:
        hourly_usage_building = hourly_usage_building.reset_index()
        hourly_usage_building['timestamp'] = hourly_usage_building['index'].apply(lambda x: datetime.datetime.timestamp(x))
        hourly_usage_building['occupied'] = hourly_usage_building.timestamp.apply(lambda x: schedule_object.is_occupied(x, resolution='exact'))
        unoccupied_avg_usage_kwh = hourly_usage_building.groupby(['occupied']).mean()[['electric_usage_kwh']].loc[False].values[0]
        occupied_avg_usage_kwh = hourly_usage_building.groupby(['occupied']).mean()[['electric_usage_kwh']].loc[True].values[0]
        night_to_day_elec_use_ratio = unoccupied_avg_usage_kwh / occupied_avg_usage_kwh
        
    return night_to_day_elec_use_ratio      
    

all_buildings = server.buildings()
org_df = pd.DataFrame(all_buildings)
# change organizatioon data into pandas columns and then merge back in to original dataframe
organizations = org_df.organizations.apply(lambda x: pd.Series(x))
org_columns = organizations[0].apply(pd.Series)
org_columns = org_columns.rename(columns={0: 'organization_id',
                                         1: 'organization_name'})
org_df = pd.merge(org_df, org_columns, how='left',
                 left_index=True, right_index=True)

if org_id == 0:
    organization_name = 'all organizations on the server found at ' + server_web_address
else:
    organization_name = org_df.query("organization_id == @org_id").organization_name.iloc[0]

def check_data_for_buildings_in_org(org_df, org_id):
    ''' Checks a dataframe with the building metadata for an entire organization to
    determine if there is sufficient data for each building to calculate an EUI/ECI. 
    Returns a list of building ids for all buildings in the organization with sufficient
    data.'''
    if org_id == 0:
        org_df = org_df
    else:
        org_df = org_df.query("organization_id == @org_id")
        
    buildings_with_sufficient_data = pd.DataFrame(columns=['building_id', 'cost_data'])
    counter = 0
    for building_id in org_df.id.unique():
        
        building_df = org_df.query("id == @building_id")
        
        insufficient_data = 0 
        
        if building_df.floor_area.isna().iloc[0]:
            insufficient_data = 1 
        elif building_df['fuel_ids'].empty or building_df['fuel_ids'].iloc[0] == '':
            insufficient_data = 1 
        elif building_df['electric_ids'].empty or building_df['electric_ids'].iloc[0] == '':
            insufficient_data = 1 
        
        if insufficient_data == 0:
            cost_data = 1

            if building_df.fuel_rate.iloc[0] is None:
                cost_data = 0
            elif building_df.electric_rate.iloc[0] is None:
                cost_data = 0
            
            buildings_with_sufficient_data.at[counter, 'building_id'] = building_id
            buildings_with_sufficient_data.at[counter, 'cost_data'] = cost_data
            counter += 1
            
    return buildings_with_sufficient_data

buildings_with_data = check_data_for_buildings_in_org(org_df, org_id)

if buildings_with_data.empty:
    error_message = 'Buildings in this organization have insufficient data for comparison at this point in time.'
    raise RuntimeError(error_message)

for idx, row in buildings_with_data.iterrows():
    building_id = row['building_id']
    building_df = org_df.query("id == @building_id")
    electric_dates = server.sensor_readings(building_df['electric_ids'].values[0]).index
    timedelta = relativedelta(electric_dates.max(), electric_dates.min())
    months_of_data = timedelta.years * 12 + timedelta.months
    buildings_with_data.at[idx, 'months_of_data'] = months_of_data

buildings_with_data['at_least_three_months'] = np.where(buildings_with_data.months_of_data >= 3, 1, 0)
buildings_with_data['at_least_six_months'] = np.where(buildings_with_data.months_of_data >= 6, 1, 0)
buildings_with_data['at_least_one_year'] = np.where(buildings_with_data.months_of_data >= 12, 1, 0)

if buildings_with_data.at_least_one_year.sum() >= (buildings_with_data.shape[0]*0.4):
    months_of_data = 12
elif buildings_with_data.at_least_six_months.sum() >= (buildings_with_data.shape[0]*0.4):
    months_of_data = 6
elif buildings_with_data.at_least_three_months.sum() >= (buildings_with_data.shape[0]*0.4):
    months_of_data = 3 
else: 
    error_message = 'The majority of builings in this organization do not appear to have 3 months of data.'
    raise RuntimeError(error_message)  

if buildings_with_data.cost_data.sum() != buildings_with_data.shape[0]:
    all_building_euis = []
    for building_id_i in buildings_with_data.building_id.unique():
        building_df = calculate_eui(server, 
                                    building_id_i, 
                                    datetime.datetime.now() - relativedelta(months=months_of_data), 
                                    datetime.datetime.now())
        building_df['last_months_night_to_day_electric_use_ratio'] = calculate_unoccupied_elec_ratio(server,
                                                                                                     building_id_i,
                                                                                                     datetime.datetime.now() - relativedelta(months=1),
                                                                                                     datetime.datetime.now())
        all_building_euis.append(building_df)
    all_building_df = pd.concat(all_building_euis, axis=1)
elif buildings_with_data.cost_data.sum() == buildings_with_data.shape[0] and buildings_with_data.shape[0] > 0:
    all_building_euis_ecis = []
    for building_id_i in buildings_with_data.building_id.unique():
        building_df = calculate_eui_eci(server, 
                                        building_id_i,
                                        datetime.datetime.now() - relativedelta(months=months_of_data), 
                                        datetime.datetime.now())
        building_df['last_months_night_to_day_electric_use_ratio'] = calculate_unoccupied_elec_ratio(server,
                                                                                             building_id_i,
                                                                                             datetime.datetime.now() - relativedelta(months=1),
                                                                                             datetime.datetime.now())
        all_building_euis_ecis.append(building_df)
    all_building_df = pd.concat(all_building_euis_ecis, axis=1)

all_building_df = all_building_df.transpose()

all_building_df = all_building_df.rename(columns={'monthly_electric_usage_kwh':'annual_electric_usage_kwh',
                                                 'monthly_fuel_ccf':'annual_fuel_use_ccf',
                                                 })
all_building_df['annual_fuel_usage_mmbtu'] = all_building_df.fuel_usage / 1000000

twelve_diverging_hues = ['#a6cee3','#1f78b4','#b2df8a',
                         '#33a02c', '#fb9a99', '#e31a1c',
                         '#fdbf6f', '#ff7f00', '#cab2d6',
                         '#6a3d9a', '#ffff99', '#b15928']

five_diverging_hues = ['#a6cee3','#1f78b4','#b2df8a',
                       '#33a02c','#fb9a99']

all_building_df['color'] = ''

if all_building_df.shape[0] <= 5:
    counter = 0
    for i_color in five_diverging_hues:
        if counter < all_building_df.shape[0]:
            all_building_df.at[counter, 'color'] = i_color
            counter += 1
else:
    counter = 0
    for i_color in twelve_diverging_hues:
        if counter < all_building_df.shape[0]:
            if counter < 12:
                all_building_df.at[counter, 'color'] = i_color
                counter += 1
            else:
                twelve_diverging_hues.append(twelve_diverging_hues)
                all_building_df.at[counter, 'color'] = i_color
                counter += 1

md = "# Energy and cost comparisons of " + organization_name + ' buildings over the past ' + str(months_of_data) + ' months'

##############################################################################################

Markdown(md)

In [ ]:
all_building_df = all_building_df.sort_values('electric_eui', ascending=True)

bar1 = go.Bar(x=all_building_df.electric_eui,
              y=all_building_df.building_name, 
              orientation='h',
              text=all_building_df.building_name,
              textposition='auto', 
              marker_color=all_building_df.color)

data = [bar1]

layout = dict(title='Electricity Use Per Square Foot for ' + organization_name + ' buildings over the past ' + str(months_of_data) + ' months',
              xaxis=dict(title='Electric EUI (kWh per square foot)'), 
             yaxis=dict(showticklabels=False))

fig = go.Figure(dict(data=data, layout=layout))

fig.show()

In [ ]:
all_building_df = all_building_df.sort_values('fuel_eui', ascending=True)

bar1 = go.Bar(x=all_building_df.fuel_eui,
              y=all_building_df.building_name, 
              orientation='h',
              text=all_building_df.building_name,
              textposition='auto', 
              marker_color=all_building_df.color)

data = [bar1]

layout = dict(title='Fuel Use Per Square Foot for ' + organization_name + ' buildings over the past ' + str(months_of_data) + ' months',
              xaxis=dict(title='Fuel EUI (BTUs per square foot)'), 
             yaxis=dict(showticklabels=False))

fig = go.Figure(dict(data=data, layout=layout))

fig.show()

In [ ]:
if 'eci' in all_building_df.columns:
    all_building_df = all_building_df.sort_values('eci', ascending=True)

    bar1 = go.Bar(x=all_building_df.eci,
                  y=all_building_df.building_name, 
                  orientation='h',
                  text=all_building_df.building_name,
                  textposition='auto', 
                  marker_color=all_building_df.color)

    data = [bar1]

    layout = dict(title='Energy Cost Per Square Foot for ' + organization_name + ' buildings over the past ' + str(months_of_data) + ' months',
                  xaxis=dict(title='ECI ($ per square foot)'), 
                 yaxis=dict(showticklabels=False))

    fig = go.Figure(dict(data=data, layout=layout))

    fig.show()

In [ ]:
all_building_df = all_building_df.sort_values('annual_electric_usage_kwh', ascending=True)

bar1 = go.Bar(x=all_building_df.annual_electric_usage_kwh,
              y=all_building_df.building_name, 
              orientation='h',
              text=all_building_df.building_name,
              textposition='auto', 
              marker_color=all_building_df.color)

data = [bar1]

layout = dict(title='Total Electricity Consumption for ' + organization_name + ' buildings over the past ' + str(months_of_data) + ' months',
              xaxis=dict(title='Electricity Consumption (kWh)'), 
             yaxis=dict(showticklabels=False))

fig = go.Figure(dict(data=data, layout=layout))

fig.show()

In [ ]:
all_building_df = all_building_df.sort_values('annual_fuel_usage_mmbtu', ascending=True)

bar1 = go.Bar(x=all_building_df.annual_fuel_usage_mmbtu,
              y=all_building_df.building_name, 
              orientation='h',
              text=all_building_df.building_name,
              textposition='auto', 
              marker_color=all_building_df.color)

data = [bar1]

layout = dict(title='Total Fuel Consumption for ' + organization_name + ' buildings over the past ' + str(months_of_data) + ' months',
              xaxis=dict(title='Fuel Consumption (MMBTU per year)'), 
             yaxis=dict(showticklabels=False))

fig = go.Figure(dict(data=data, layout=layout))

fig.show()

In [ ]:
if 'total_monthly_costs' in all_building_df.columns:

    all_building_df = all_building_df.sort_values('total_monthly_costs', ascending=True)

    bar1 = go.Bar(x=all_building_df.total_monthly_costs,
                  y=all_building_df.building_name, 
                  orientation='h',
                  text=all_building_df.building_name,
                  textposition='auto', 
                  marker_color=all_building_df.color)

    data = [bar1]

    layout = dict(title='Total energy costs for ' + organization_name + ' buildings over the past ' + str(months_of_data) + ' months',
                  xaxis=dict(title='Energy Costs ($)'), 
                 yaxis=dict(showticklabels=False))

    fig = go.Figure(dict(data=data, layout=layout))

    fig.show()

In [ ]:
all_building_df = all_building_df.query("last_months_night_to_day_electric_use_ratio == last_months_night_to_day_electric_use_ratio")
all_building_df = all_building_df.sort_values('last_months_night_to_day_electric_use_ratio', ascending=True)

bar1 = go.Bar(x=all_building_df.last_months_night_to_day_electric_use_ratio,
              y=all_building_df.building_name, 
              orientation='h',
              text=all_building_df.building_name,
              textposition='auto', 
              marker_color=all_building_df.color)

data = [bar1]

layout = dict(title='Comparison of unoccupied to occupied electric consumption ratios for ' + organization_name + ' buildings over the past month',
              xaxis=dict(title='Ratio of average unoccupied electricity consumption to average occupied electricity consumption'), 
             yaxis=dict(showticklabels=False))

fig = go.Figure(dict(data=data, layout=layout))

fig.show()